In [1]:
#Mount my drive- run the code, go to the link, accept.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/My Drive/OwnJupyterNotebooks/")
os.getcwd()

'/content/gdrive/My Drive/OwnJupyterNotebooks'

In [0]:
#!pip install --pre -U tensorflow


### Importing Libraries


In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.6.0
GPU is NOT AVAILABLE


In [0]:
# Split data int training, validation, and test data sets - training set of 25000 is split into 60%, 15000, training data, 
# and 40%, 10000, validation data.
train_validation_split = tfds.Split.TRAIN.subsplit([6,4])
(train_data, validation_data), test_data = tfds.load(
  name='imdb_reviews',
  split=(train_validation_split, tfds.Split.TEST),
  as_supervised=True)

In [8]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print(train_examples_batch)

tf.Tensor(
[b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a journalist rather than a novelist. It'

In [9]:
train_labels_batch

<tf.Tensor: id=220, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

### Building the Model

In [10]:
# Creating embedding layers with TF Hub
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1" # pre-trained text embedding model that converts strings to vectors
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=400, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

#### Activation Functions: The first shaping of input into output, non-linear, fed into more layers.  It acts on the sum of (weights X inputs) plus biases.<br>
Identity<br>
Binary Step: y=1 if x > 0, not good for optimizing since the gradient is 0.<br>
Sigmoid: y=1/(1+e^-x) output range between 0 and 1 and the derivative isn't constant or zero. Optimization possible but still vanishing gradient.<br>
Tanh: centers the sigmoid around 0 so negative outputs are possible but vanishing gradient.<br>
ReLU: y=x, x>0, 0, x<0, widely used, doesn't activate all neurons at a time.<br>
Leaky ReLU: f(x)= ax, x<0
= x, x>=0, improved ReLU, removes non-zero gradient in the derivative-eliminated dead neurons.<br>
Softmax: type of sigmoid function but good for multiple classes.<br>
Only use ReLU's in the hidden layers.

In [11]:
# Full Model 
model = tf.keras.Sequential() # using keras Sequential() model api
model.add(hub_layer) #Using the add() method to add the embedding layer
model.add(tf.keras.layers.Dense(16, activation='relu')) # specifying dense layers and activation functions
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # Using sigmoid because output is binary

model.summary() # calling a summary of inputs


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Specifying optimizer and loss function, and metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Training the Model

In [14]:
# Specifying  number of full iterations over models (epochs)
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1) 

Epoch 1/20
30/30 [==============================] - 8s 251ms/step - loss: 0.1640 - accuracy: 0.9444 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 7s 250ms/step - loss: 0.1560 - accuracy: 0.9497 - val_loss: 0.2997 - val_accuracy: 0.8763
Epoch 3/20
30/30 [==============================] - 8s 251ms/step - loss: 0.1427 - accuracy: 0.9537 - val_loss: 0.3028 - val_accuracy: 0.8746
Epoch 4/20
30/30 [==============================] - 8s 251ms/step - loss: 0.1383 - accuracy: 0.9556 - val_loss: 0.3020 - val_accuracy: 0.8766
Epoch 5/20
30/30 [==============================] - 8s 254ms/step - loss: 0.1300 - accuracy: 0.9599 - val_loss: 0.3062 - val_accuracy: 0.8747
Epoch 6/20
30/30 [==============================] - 8s 251ms/step - loss: 0.1192 - accuracy: 0.9632 - val_loss: 0.3060 - val_accuracy: 0.8767
Epoch 7/20
30/30 [==============================] - 7s 248ms/step - loss: 0.1135 - accuracy: 0.9673 - val_loss: 0.3094 - val_accuracy: 0.876

### Evaluating the Model

In [15]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.4086 - accuracy: 0.8558
loss: 0.409
accuracy: 0.856
